In [40]:
from BigDFT import Datasets as D, Calculators as C, Inputfiles as I, Logfiles as lf
from BigDFT.Database import Molecules
import numpy as np
import sys,os
sys.path.insert(0,'../')
import StatPol as SP

basepath = os.getcwd()
basepath

'/home/marco/Data/RICERCA/DFT AND MANY BODY/LR MOLECULES/Statical polarizability/Analysis/Calculations'

In [2]:
omp = 2
mpi_run = 'mpirun -np 4'

# Workflow

This notebook contains the complete workflow to extract the statical polarizability of the HG dataset.

In this workflow we choose a set of values for the intensity of the field and, for each value, we perform a convergence analysis on the value of rmult

## Construction of the relevant tools

Define the functions needed to perform the workflow

In [3]:
# Service functions 

def change_workdir(molecule,study,datadir):
    """
    Ensure that the path Data_directory/molecule exists and
    move there. 
    """
    from futile.Utils import ensure_dir
    for d in [datadir,molecule]:
        ensure_dir(d)
        os.chdir(d)
    path='-'.join(study)
    ensure_dir(path)
    return '-'.join([molecule,path]),path

def get_psp(molecule):
    """
    Check if the all the atoms of molecule have the nlcc of the type nlss_aw and nlcc_ss. 
    If it is true the corresponding pseudo is added to the list of psp to be performed, otherwise 
    only hgh_k is included.
    """
    from os import path as p
    import Routines as R
    possible_psp=['nlcc_aw','nlcc_ss']
    to_do=[True,True]
    for at in R.get_atoms(molecule):
        psp_available = [p.isfile(p.join('psp_'+psp,'psppar.'+at)) for psp in possible_psp]
        to_do = [ a and b for a,b in zip(to_do,psp_available)]
    return ['hgh_k'] + [psp for yes,psp in zip(to_do,possible_psp) if yes]

def split_dataset(dataset,length=10):
    """
    Split a list of molecules into a list of list with length elements. 
    """
    splitted = [[] for num in range(int(len(dataset)/length)+1)]
    for ind,mol in enumerate(dataset):
        splitted[int(ind/length)].append(mol)
    return splitted

In [4]:
# Tools for the ground state

def get_converged_input_energy(dataset,rtol,atol):
    data=dataset.seek_convergence(atol=atol,rtol=rtol,attribute='energy')
    return dataset.runs[dataset.names.index(D.name_from_id(data[0]))]['input'],data[1]

def find_gs_domain(label,rtol,atol,path,input,posinp,code):
    """
    Use the seek_convergence method of Dataset to perform a convergence procedure on
    the rmult value for a gs computation.
    """
    crmult=input['dft']['rmult'][0]
    rmult_fine=input['dft']['rmult'][1]
    rmult_list=map(float,range(int(crmult),11))
    seek_for_rmult = D.Dataset(label=label+'(GS)',run_dir=path,posinp=posinp)
    for rm in rmult_list:
        input.set_rmult(coarse=rm,fine=rmult_fine)
        seek_for_rmult.append_run(id={'rmult':rm},runner=code,input=input)
    input_gs,log_gs=get_converged_input_energy(seek_for_rmult,rtol,atol)
    return {'input_gs':input_gs, 'dataset_gs': seek_for_rmult,'log_gs':log_gs}

def gs_study(molecule,study,code,options):
    """"
    Workflow for the convergence analysis of the ground state.
    
    Args : 
       molecule (str) : name of the molecule
       study (touple) : the couple (xc,psp)
       code (runner)  : instance of SystemCalculator
       options (dict) : dictionary with the computational options
    """
    import Routines as R
    hgrids=options.get('hgrids')
    rmult_fine=options.get('rmult_fine')
    wf_convergence=options.get('wf_convergence')
    crmult_start=options.get('crmult',4.0)
    rtol=options.get('rtol_gs',10*wf_convergence)
    atol=options.get('atol_gs',1.e-3)
    reference_results=options.get('reference_data')
    datadir=options.get('data_directory','Data')

    initial_dir=os.path.abspath(os.path.dirname('.'))
    label,path=change_workdir(molecule,study,datadir)
    
    posinp=Molecules.Molecule(molecule)
    
    inp = I.Inputfile()
    inp.set_hgrid(hgrids)
    inp.set_rmult(coarse=crmult_start,fine=rmult_fine)
    R.set_xc[study](inp,molecule,datadir)
    inp.set_wavefunction_convergence(wf_convergence)
    mpol=int(reference_results[molecule]['mpol_ref'])-1   
    if mpol >0: inp.spin_polarize(mpol)
    data= find_gs_domain(label,10*wf_convergence,atol,path,inp,posinp,code)
    os.chdir(initial_dir)
    
    data['molecule']=molecule
    data['posinp']=posinp
    data['study']=study
    return data 

In [5]:
# Tools for alpha
def extract_statical_polarizability(label,rtol,atol,path,input_for_alpha,posinp,code,ref):
    
    # seek for the field convergence
    ints=[1e-2,5e-3,1e-3,5e-4,1e-4]
    alpha_field=D.Dataset(label=label+'(field)',run_dir=path)
    for f in ints:
        alpha_field.append_run(id={'f':f},runner=SP.build_alpha_dataset(input=input_for_alpha,posinp=posinp,run_dir=path,runner=code,intensity=f))
    try:
        data_field=alpha_field.seek_convergence(rtol=rtol,atol=atol)
        intensity=data_field[0]['f']
    except:
        print ('Convergence in field not reached')
        intensity=ints[-1]
    
    # seek for the rmult convergence
    rmult_fine=input_for_alpha['dft']['rmult'][1]
    rmult_list=map(float,range(int(input_for_alpha['dft']['rmult'][0]),11))
    alpha_rmult=D.Dataset(label=label+'(domain)',run_dir=path)
    for rm in rmult_list:
        input_for_alpha.set_rmult(coarse=rm,fine=rmult_fine)
        alpha_rmult.append_run(id={'f':intensity,'rmult':rm},runner=SP.build_alpha_dataset(input=input_for_alpha,posinp=posinp,run_dir=path,runner=code,intensity=intensity))
    try:
        import numpy as np
        data_final=alpha_rmult.seek_convergence(rtol=rtol,atol=atol)
        AuToA = 0.5291772085**3
        alpha_ref_ii = np.array(ref)/AuToA
        eps = 100.0 * (data_final[1]-alpha_ref_ii)/alpha_ref_ii
        print ('Relative difference in %',eps.tolist())
    except LookupError as e:
        print ('Convergence in domain not reached',e)
        data_final=None
    return {'alpha_convergence': data_final, 'dataset_field': alpha_field, 'dataset_rmult': alpha_rmult}

def extract_statical_polarizability_linearity(label,rtol,atol,path,input_for_alpha,posinp,code,ref):
    
    ints=[1e-2,5e-3,1e-3,5e-4,1e-4]
       
    results = {}
    results['alpha_values'] = {}
    rmult_fine=input_for_alpha['dft']['rmult'][1]
    rmult_list=map(float,range(int(input_for_alpha['dft']['rmult'][0]),11))
    for intensity in ints:
    
        # seek for the rmult convergence
        alpha_rmult=D.Dataset(label=label+'(domain)',run_dir=path)
        for rm in rmult_list:
            input_for_alpha.set_rmult(coarse=rm,fine=rmult_fine)
            alpha_rmult.append_run(id={'f':intensity,'rmult':rm},\
                runner=SP.build_alpha_dataset(input=input_for_alpha,posinp=posinp,run_dir=path,\
                            runner=code,intensity=intensity))
    
        try:
            import numpy as np
            data_final=alpha_rmult.seek_convergence(rtol=rtol,atol=atol)
            AuToA = 0.5291772085**3
            alpha_ref_ii = np.array(ref)/AuToA
            eps = 100.0 * (data_final[1]-alpha_ref_ii)/alpha_ref_ii
            print ('Relative difference in %',eps.tolist())
        except LookupError as e:
            print ('Convergence in domain not reached',e)
            data_final=None
        
        results['alpha_values'][intensity] = {'alpha_convergence': data_final, 'dataset_rmult': alpha_rmult}
    return results

def alpha_study(gs_data,code,options):
    """"
    Workflow for the convergence analysis for the computation of alpha.
    
    Args : 
       gs_data        : output of gs_study
       code (runner)  : instance of SystemCalculator
       options (dict) : dictionary with the computational options
    """
    import copy
    xc_conversion={'lda_pw': 'lda-SPW92', 'pbe': 'pbe', 'pbe0': 'pbe0'}
    rtol=options.get('rtol',1.e-2)
    atol=options.get('atol',1.e-3)
    reference_results=options.get('reference_data')
    datadir=options.get('data_directory','Data')
    study=gs_data['study']
    molecule=gs_data['molecule']
    posinp=gs_data['posinp']
    xc=study[0]
    ref_data=reference_results[molecule][xc_conversion[xc]]
    
    initial_dir=os.path.abspath(os.path.dirname('.'))
    label,path=change_workdir(molecule,study,datadir)
    input_gs=copy.deepcopy(gs_data['input_gs'])
        
    #data=extract_statical_polarizability(label,rtol,atol,path,input_gs,posinp,code,ref=ref_data)
    data=extract_statical_polarizability_linearity(label,rtol,atol,path,input_gs,posinp,code,ref=ref_data)
    os.chdir(initial_dir)
    
    return data

In [6]:
def data_to_save(data,options):
    """
    Define the dictionary to be saved on file for data analysis
    """
    from copy import deepcopy
    res = {}
    for s,values in data.iteritems():
        res[s] = {}
        if not (values['alpha_convergence'] is None):
            res[s]['alpha_convergence'] = (values['alpha_convergence'][0],values['alpha_convergence'][1].tolist())
        else : 
            res[s]['alpha_convergence'] = values['alpha_convergence']
        
        res[s]['posinp'] = {}
        pos = values['posinp']
        for key in pos:
            res[s]['posinp'][key] = pos[key]
        
        res[s]['input_gs'] = {}
        inp = values['input_gs']
        for key in inp:
            res[s]['input_gs'][key] = inp[key]
        
    save_options = deepcopy(options)
    save_options.pop('reference_data')
    res['options'] = save_options
    return res

def data_to_save_linearity(data,options):
    """
    Define the dictionary to be saved on file for data analysis
    """
    from copy import deepcopy
    res = {}
    for s,values in data.iteritems():
        res[s] = {}
        res[s]['alpha_values'] = {}
        alpha_values = values['alpha_values']
        for int,v in alpha_values.iteritems():
            res[s]['alpha_values'][int] = {}
            if not (v['alpha_convergence'] is None):
                res[s]['alpha_values'][int]['alpha_convergence'] = (v['alpha_convergence'][0],v['alpha_convergence'][1].tolist())
            else: 
                res[s]['alpha_values'][int]['alpha_convergence'] = values['alpha_convergence']
        
        res[s]['posinp'] = {}
        pos = values['posinp']
        for key in pos:
            res[s]['posinp'][key] = pos[key]
        
        res[s]['input_gs'] = {}
        inp = values['input_gs']
        for key in inp:
            res[s]['input_gs'][key] = inp[key]
        
    save_options = deepcopy(options)
    save_options.pop('reference_data')
    res['options'] = save_options
    return res

## Single study analysis - linearity test

Build the dataset using the HG data written in hg_data.yaml. To each molecule of the dataset associate a set of studies=(xc,psp)

In [7]:
import yaml
HG_data=yaml.load(open('../HG Dataset/hg_data.yaml'))

In [8]:
code=C.SystemCalculator(omp=omp,mpi_run=mpi_run,skip=True,verbose=False)

Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft


In [9]:
options={'wf_convergence':1.e-6,'hgrids':0.3,'rmult_fine':9.0,'rtol':1.e-3,'atol':1.e-3,\
         'reference_data':HG_data,'data_directory':'Data'}

We can perform a single study with this workflow

In [27]:
study_data = {}

In [28]:
molecule = 'NO'#'CO'
xc = 'pbe'
psp = 'nlcc_ss'#'nlcc_aw'

In [29]:
HG_data[molecule]

{'CCSD(T)': [1.753, 1.753, 2.283],
 'lda-SPW92': [1.872, 1.872, 2.358],
 'lda-Slater': [1.993, 1.993, 2.49],
 'mpol_ref': '1',
 'pbe': [1.856, 1.856, 2.363],
 'pbe0': [1.778, 1.778, 2.274],
 'spin_pol': 'nsp'}

In [30]:
gs_data=gs_study(molecule,(xc,psp),code,options)

Fetching results for id " {'rmult': 4.0} "
Fetching results for id " {'rmult': 5.0} "
Fetching results for id " {'rmult': 6.0} "
Convergence reached in Dataset "CO-pbe-nlcc_aw(GS)" for id " {'rmult': 5.0} "


In [32]:
#gs_data

In [33]:
study_data[(molecule,xc,psp)]=alpha_study(gs_data,code,options)
study_data[(molecule,xc,psp)].update(gs_data)

Fetching results for id " {'rmult': 5.0, 'f': 0.01} "
Fetching results for id " {'rmult': 6.0, 'f': 0.01} "
Fetching results for id " {'rmult': 7.0, 'f': 0.01} "
Fetching results for id " {'rmult': 8.0, 'f': 0.01} "
Convergence reached in Dataset "CO-pbe-nlcc_aw(domain)" for id " {'rmult': 7.0, 'f': 0.01} "
('Relative difference in %', [0.33302443490052186, 0.33302443490052186, -0.14013663030754425])
Fetching results for id " {'rmult': 5.0, 'f': 0.005} "
Fetching results for id " {'rmult': 6.0, 'f': 0.005} "
Fetching results for id " {'rmult': 7.0, 'f': 0.005} "
Fetching results for id " {'rmult': 8.0, 'f': 0.005} "
Convergence reached in Dataset "CO-pbe-nlcc_aw(domain)" for id " {'rmult': 7.0, 'f': 0.005} "
('Relative difference in %', [0.1266037733590598, 0.1266037733590598, -0.35506403390880337])
Fetching results for id " {'rmult': 5.0, 'f': 0.001} "
Fetching results for id " {'rmult': 6.0, 'f': 0.001} "
Fetching results for id " {'rmult': 7.0, 'f': 0.001} "
Fetching results for id 

In [34]:
study_data[molecule,xc,psp].keys()

['alpha_values',
 'posinp',
 'log_gs',
 'input_gs',
 'study',
 'molecule',
 'dataset_gs']

In [35]:
bla = study_data[molecule,xc,psp]['alpha_values'][0.01]['dataset_rmult']

In [36]:
bla.ids

[{'f': 0.01, 'rmult': 5.0},
 {'f': 0.01, 'rmult': 6.0},
 {'f': 0.01, 'rmult': 7.0},
 {'f': 0.01, 'rmult': 8.0},
 {'f': 0.01, 'rmult': 9.0},
 {'f': 0.01, 'rmult': 10.0}]

In [37]:
bla.calculators[2]['calc'].runs[0]

{'input': {'dft': {'elecfield': [0.01, 0.0, 0.0],
   'gnrm_cv': 1e-06,
   'hgrids': 0.3,
   'ixc': 11,
   'rmult': [7.0, 9.0]}},
 'intensity': 0.01,
 'label': 'alpha_0.01',
 'posinp': {'positions': [{'C': [0.0, 0.0, 0.0], 'frag': ['molecule', '0']},
   {'O': [0.0, 0.0, 1.1282], 'frag': ['molecule', '0']}],
  'units': 'angstroem'},
 'run_dir': 'pbe-nlcc_aw',
 'runner': <BigDFT.Calculators.SystemCalculator instance at 0x7f887b6d5cf8>}

In [38]:
out = data_to_save_linearity(study_data,options)

In [39]:
out

{'options': {'atol': 0.001,
  'data_directory': 'Data',
  'hgrids': 0.3,
  'rmult_fine': 9.0,
  'rtol': 0.001,
  'wf_convergence': 1e-06},
 ('CO',
  'pbe',
  'nlcc_aw'): {'alpha_values': {0.0001: {'alpha_convergence': ({'f': 0.0001,
      'rmult': 7.0},
     [12.530639999999998, 12.530639999999998, 15.878449999999974])},
   0.0005: {'alpha_convergence': ({'f': 0.0005, 'rmult': 7.0},
     [12.532213, 12.532213, 15.878550000000004])},
   0.001: {'alpha_convergence': ({'f': 0.001, 'rmult': 7.0},
     [12.53276, 12.53276, 15.878894999999996])},
   0.005: {'alpha_convergence': ({'f': 0.005, 'rmult': 7.0},
     [12.540765999999998, 12.540765999999998, 15.889695000000001])},
   0.01: {'alpha_convergence': ({'f': 0.01, 'rmult': 7.0},
     [12.56662,
      12.56662,
      15.923968000000002])}}, 'input_gs': {'dft': {'gnrm_cv': 1e-06,
    'hgrids': 0.3,
    'ixc': 11,
    'rmult': [5.0, 9.0]}}, 'posinp': {'positions': [{'C': [0.0, 0.0, 0.0],
     'frag': ['molecule', '0']},
    {'O': [0.0, 0.0, 

## Data analysis

In [69]:
AuToA = 0.5291772085**3

def mean_relative_error(alpha,alpha_ref):
    """
    Provide the mean relative error
    """
    return 100.0*np.mean((alpha-alpha_ref)/alpha_ref)

In [70]:
HG_data[molecule][xc]

[1.856, 1.856, 2.363]

We build a dictionary with the mean realive errors

In [71]:
mre_data = {}
for (mol,xc,psp),data in study_data.iteritems():
    mre_data[mol,xc,psp] = {}
    for ints,alphas in data['alpha_values'].iteritems():
        alpha = np.array(alphas['alpha_convergence'][1])
        alpha_ref = np.array(HG_data[molecule][xc])/AuToA
        mre_data[mol,xc,psp][ints] = mean_relative_error(alpha,alpha_ref)

In [72]:
mre_data

{('CO', 'pbe', 'nlcc_aw'): {0.0001: -0.11135604597948508,
  0.0005: -0.10277436227124005,
  0.001: -0.09914166103649091,
  0.005: -0.03395216239689459,
  0.01: 0.17530407983116647}}